# <center>Đồ án 3: Linear Regression</center>

# Thông tin sinh viên

- Họ và tên: Nguyễn Khánh Nhân
- MSSV: 21127657
- Lớp: 21CLC02

# Import

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
# Import thêm dữ thư viện nếu cần

# Đọc dữ liệu

In [2]:
# Đọc dữ liệu bằng pandas
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Lấy các đặc trưng X và giá trị mục tiêu y cho các tập huấn luyện (train) và kiểm tra (test)
X_train = train.iloc[:, :-1]    # Dataframe (chứa các đặc trưng huấn luyện)
y_train = train.iloc[:, -1]     # Series    (chứa 1 giá trị mục tiêu kiểm tra)

X_test = test.iloc[:, :-1]      # Dataframe (chứa các đặc trưng kiểm tra)
y_test = test.iloc[:, -1]       # Series    (chứa 1 giá trị mục tiêu kiểm tra)

# Sinh viên có thể sử dụng các khác nếu cần

# Cài đặt hàm

In [3]:
# Cài đặt các hàm cần thiết ở đây

# Sử dụng lại class đã được cung cấp trong Lab04 để thực hiện huấn luyện mô hình
class OLSLinearRegression:
    def fit(self, X, y):
        X_pinv = np.linalg.inv(X.T @ X) @ X.T   
        self.w = X_pinv @ y
        return self


    def get_params(self):
        return self.w


    def predict(self, X):
        return np.sum(self.w.ravel() * X, axis=1)   

# Class tự tạo để thực hiện phương pháp k-folds cross validation
class k_folds_cross_validation:
    def __init__(self, k: int, features: list, models: list, data):
        self.k = k
        self.models = models
        self.data = data.loc[:, features]
        
    def shuffle_data(self):
        """Xáo trộn dữ liệu trước khi split"""
        # cố định seed trước khi shuffle
        np.random.seed(42)
        # thực hiện shuffle data
        data_arr = self.data.values
        np.random.shuffle(data_arr)
        self.data_shuffled = pd.DataFrame(data_arr, columns=self.data.columns)
        
    def split_to_k_folds(self):
        """Chia dữ liệu đã được xáo trộn thành k fold bằng nhau"""
        rows = self.data_shuffled.shape[0]
        self.folds = [self.data_shuffled[i*(rows//self.k): (i+1)*(rows//self.k)] for i in range(self.k)]
        
    def cross_validation(self):
        """Thực hiện cross validation"""
        # Lưu giá trị mae của mỗi mô hình được huấn luyện trên từng fold
        self.maes_list = []
        for fold in self.folds:
            mae_for_each_model = []
            for model in self.models:
                X_train = fold.loc[:, model].to_numpy().reshape(-1, len(model))
                y_train = fold.iloc[:, -1]
                current_model = OLSLinearRegression().fit(X_train, y_train)
                y_pred = current_model.predict(X_train)
                current_mae = mae(y=y_train, y_hat=y_pred)
                mae_for_each_model.append(current_mae)
               
            self.maes_list.append(mae_for_each_model)
        # Tính giá trị mae trung bình của mỗi mô hình sau khi được huấn luyện trên k fold
        self.maes_list = np.mean(np.array(self.maes_list), axis=0)  
        
    def best_model(self):
        """Trả về mảng giá trị mae trung bình sau khi thực hiện k-fold cross validation và mô hình có giá trị mae
        trung bình nhỏ nhất"""
        return self.maes_list, np.argmin(self.maes_list)  

# Hàm tính mae được cung cấp sẵn ở Lab04
def mae(y, y_hat):
    return np.mean(np.abs(y.ravel() - y_hat.ravel()))

# Yêu cầu 1a: Sử dụng toàn bộ 11 đặc trưng đầu tiên `Gender`, `10percentage`, `12percentage`, `CollegeTier`, `Degree`, `collegeGPA`, `CollegeCityTier`, `English`, `Logical`, `Quant`, `Domain` (2 điểm) 

In [4]:
# Phần code cho yêu cầu 1a
# Lấy các dữ liệu cần thiết
features_1a = ['Gender', '10percentage', '12percentage', 'CollegeTier', 'Degree', 'collegeGPA', 'CollegeCityTier', 'English', 'Logical', 'Quant', 'Domain']
X_train_1a = train.loc[:, features_1a]
y_train_1a = train.iloc[:, -1]
X_test_1a = test.loc[:, features_1a]
y_test_1a = test.iloc[:, -1]
# Bắt đầu xây dựng model tương ứng với 11 feature đầu tiên
eleven_features_model = OLSLinearRegression().fit(X_train_1a, y_train_1a)
# In ra các hệ số tương ứng vừa huấn luyện được cho từng feature
params_1a = eleven_features_model.get_params()
for i in range(len(params_1a)):
    print(f"{features_1a[i]}: {params_1a[i]}")
# Tính giá trị dự đoán của tập kiểm tra dựa theo mô hình vừa có được
y_pred_1a = eleven_features_model.predict(X_test_1a)

Gender: -22756.512821061042
10percentage: 804.5031556737734
12percentage: 1294.6545652522814
CollegeTier: -91781.89753048787
Degree: 23182.388679172
collegeGPA: 1437.5486723052477
CollegeCityTier: -8570.661984852592
English: 147.85829929012567
Logical: 152.88847647987103
Quant: 117.22184593042903
Domain: 34552.28622082775


In [5]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra
mae_1a = mae(y=y_test_1a, y_hat=y_pred_1a)
print("Độ lỗi tuyệt đối trung bình (MAE - Mean Absolute Error) của mô hình ở câu 1a: ", mae_1a)

Độ lỗi tuyệt đối trung bình (MAE - Mean Absolute Error) của mô hình ở câu 1a:  104863.77754033124


Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012):
$$\text{Salary} = -22756.513 \times \text{Genger} + 804.503 \times \text{10percentage} + 1294.655 \times \text{12percentage}\\ - 91781.897 \times \text{CollegeTier} + 23182.389 \times \text{Degree} + 1437.549 \times \text{collegeGPA} - 8570.662 \times \text{CollegeCityTier} \\ + 147.858 \times \text{English} + 152.888 \times \text{Logical} + 117.222 \times \text{Quant} + 34552.286 \times \text{Domain} $$

Độ lỗi tuyệt đối trung bình MAE trên tập kiểm tra của mô hình ở câu 1a (làm tròn đến 3 chữ số thập phân):
$$ \text{MAE} =  104863.777$$

# Yêu cầu 1b: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng tính cách với các đặc trưng tính cách gồm `conscientiousness`, `agreeableness`, `extraversion`, `nueroticism`, `openess_to_experience`, tìm mô hình cho kết quả tốt nhất (1 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [6]:
# Phần code cho yêu cầu 1b
# Tìm ra đặc trưng tốt nhất
# In ra các kết quả cross-validation như yêu cầu
features_1b = ['conscientiousness', 'agreeableness', 'extraversion', 'nueroticism' , 'openess_to_experience', 'Salary']
models_1b = [['conscientiousness'], ['agreeableness'], ['extraversion'], ['nueroticism'], ['openess_to_experience']]
kfcv_er_1b = k_folds_cross_validation(k=8, features=features_1b, models=models_1b, data=train)
kfcv_er_1b.shuffle_data()
kfcv_er_1b.split_to_k_folds()
kfcv_er_1b.cross_validation()
avg_mae_of_each_model, index_best_feature = kfcv_er_1b.best_model()
for i in range(avg_mae_of_each_model.shape[0]):
    print(f"Mae cho mô hình với đặc trưng {features_1b[i]} từ k-fold Cross Validation (lấy trung bình): {avg_mae_of_each_model[i]}")
print(f"Mô hình tốt nhất là mô hình với feature: {features_1b[index_best_feature]}")

Mae cho mô hình với đặc trưng conscientiousness từ k-fold Cross Validation (lấy trung bình): 304945.9543582987
Mae cho mô hình với đặc trưng agreeableness từ k-fold Cross Validation (lấy trung bình): 299281.0894100772
Mae cho mô hình với đặc trưng extraversion từ k-fold Cross Validation (lấy trung bình): 305733.2833096464
Mae cho mô hình với đặc trưng nueroticism từ k-fold Cross Validation (lấy trung bình): 297666.2754969283
Mae cho mô hình với đặc trưng openess_to_experience từ k-fold Cross Validation (lấy trung bình): 302362.6759361781
Mô hình tốt nhất là mô hình với feature: nueroticism


- Báo cáo **5 kết quả (mae) tương ứng cho 5 mô hình** từ k-fold Cross Validation (lấy trung bình)
<center>

| STT | Mô hình với 1 đặc trưng |   MAE    |
|:---:|:-----------------------:|:--------:|
|  1  | conscientiousness       |304945.954|
|  2  | agreeableness           |299281.089|
|  3  | extraversion            |305733.283|
|  **4**  | **neuroticism**             |**297666.275**|
|  5  | openness_to_experience  |302362.676|

</center>

In [7]:
# Lấy các dữ liệu cần thiết
X_train_1b = train.loc[:, features_1b[index_best_feature]].to_numpy().reshape(-1, 1)
y_train_1b = train.iloc[:, -1]
X_test_1b = test.loc[:, features_1b[index_best_feature]].to_numpy().reshape(-1, 1)
y_test_1b = test.iloc[:, -1]
# Huấn luyện lại mô hình best_personality_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện
best_personality_feature_model = OLSLinearRegression().fit(X_train_1b, y_train_1b)
# In ra các hệ số tương ứng vừa huấn luyện được cho từng feature
params_1b = best_personality_feature_model.get_params()
print(f"{features_1b[index_best_feature]}: {params_1b[0]}")
# Tính giá trị dự đoán của tập kiểm tra dựa theo mô hình vừa có được
y_pred_1b = best_personality_feature_model.predict(X_test_1b)

nueroticism: -56546.30375317799


In [8]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_personality_feature_model
mae_1b = mae(y=y_test_1b, y_hat=y_pred_1b)
print("Độ lỗi tuyệt đối trung bình (MAE - Mean Absolute Error) của mô hình ở câu 1b: ", mae_1b)

Độ lỗi tuyệt đối trung bình (MAE - Mean Absolute Error) của mô hình ở câu 1b:  291019.693226953


Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012):
$$\text{Salary} = -56546.304 \times nueroticism$$
Độ lỗi tuyệt đối trung bình MAE trên tập kiểm tra của mô hình tốt nhất ở câu 1b (làm tròn đến 3 chữ số thập phân):
$$ \text{MAE} =  291019.693$$

# Yêu cầu 1c: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng `English`, `Logical`, `Quant`, tìm mô hình cho kết quả tốt nhất (1 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [9]:
# Phần code cho yêu cầu 1c
# Tìm ra đặc trưng tốt nhất
# In ra các kết quả cross-validation như yêu cầu
features_1c = ['English', 'Logical', 'Quant', 'Salary']
models_1c = [['English'], ['Logical'], ['Quant']]
kfcv_er_1c = k_folds_cross_validation(k=8, features=features_1c,models=models_1c ,data=train)
kfcv_er_1c.shuffle_data()
kfcv_er_1c.split_to_k_folds()
kfcv_er_1c.cross_validation()

avg_mae_of_each_model, index_best_feature = kfcv_er_1c.best_model()
for i in range(avg_mae_of_each_model.shape[0]):
    print(f"Mae cho mô hình với đặc trưng {features_1c[i]} từ k-fold Cross Validation (lấy trung bình): {avg_mae_of_each_model[i]}")
print(f"Mô hình tốt nhất là mô hình với feature: {features_1c[index_best_feature]}")

Mae cho mô hình với đặc trưng English từ k-fold Cross Validation (lấy trung bình): 121714.64555974925
Mae cho mô hình với đặc trưng Logical từ k-fold Cross Validation (lấy trung bình): 120323.4811155938
Mae cho mô hình với đặc trưng Quant từ k-fold Cross Validation (lấy trung bình): 117863.43386526816
Mô hình tốt nhất là mô hình với feature: Quant


- Báo cáo **3 kết quả tương ứng cho 3 mô hình** từ k-fold Cross Validation (lấy trung bình)
	
	<center>

	| STT | Mô hình với 1 đặc trưng | MAE  |
	|:---:|:-----------------------:|:----:|
	|  1  | English			        |121714.645      |
	|  2  | Logical		            |120323.481      |
	|  **3**  | **Quant**		            |**117863.434**      |

	</center>

In [10]:
# Lấy các dữ liệu cần thiết
X_train_1c = train.loc[:, features_1c[index_best_feature]].to_numpy().reshape(-1, 1)
y_train_1c = train.iloc[:, -1]
X_test_1c = test.loc[:, features_1c[index_best_feature]].to_numpy().reshape(-1, 1)
y_test_1c = test.iloc[:, -1]
# Huấn luyện lại mô hình best_skill_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện  
best_skill_feature_model = OLSLinearRegression().fit(X_train_1c, y_train_1c)
# In ra các hệ số tương ứng vừa huấn luyện được cho từng feature
params_1c = best_skill_feature_model.get_params()
print(f"{features_1c[index_best_feature]}: {params_1c[0]}")
# Tính giá trị dự đoán của tập kiểm tra dựa theo mô hình vừa có được
y_pred_1c = best_skill_feature_model.predict(X_test_1c)

Quant: 585.8953808404565


In [11]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_skill_feature_model
mae_1c = mae(y=y_test_1c, y_hat=y_pred_1c)
print("Độ lỗi tuyệt đối trung bình (MAE - Mean Absolute Error) của mô hình ở câu 1c: ", mae_1c)

Độ lỗi tuyệt đối trung bình (MAE - Mean Absolute Error) của mô hình ở câu 1c:  106819.57761989674


Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012):
$$\text{Salary} = 585.895 \times Quant$$
Độ lỗi tuyệt đối trung bình MAE trên tập kiểm tra của mô hình tốt nhất ở câu 1c (làm tròn đến 3 chữ số thập phân):
$$ \text{MAE} =  106819.578$$

# Yêu cầu 1d: Sinh viên tự xây dựng mô hình, tìm mô hình cho kết quả tốt nhất (3 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ $m$ mô hình mà sinh viên thiết kế

## Xây dựng mô hình
- Trong dữ liệu của chúng ta có rất nhiều `feature X (biến độc lập)` dùng để dự đoán cho giá trị cần tìm `biến phụ thuộc y`, tuy nhiên việc sử dụng tât cả các `biến độc lập X` trong một mô hình là không phải một điều hay vì một trong số chúng có thể là `garbage feature`. Các `garbage feature` có thể làm cho mô hình của ta trở nên tệ hơn vì vậy khi xây dựng mô hình hồi quy tuyến tính cần xem xét kĩ nên giữ hay loại bỏ các `biến độc lập` nào để giúp cho mô hình của ta tốt nhất có thể.
- Có rất nhiều phương pháp để xây dựng một mô hình hồi quy tuyến tính, dưới đây sẽ là 3 phương pháp được sử dụng phổ biến và dễ thực hiện:
    - `All-in`
    - `Backward elimination`.
    - `Forward elimination`.
- Cả 2 phương pháp cuối đều sử dụng một yếu tố để đánh giá xem 1 `biến độc lập` có thực sự ảnh hưởng, đóng góp tới kết quả của biến phụ thuộc hay không đó chính là `p-value`, một giá trị mà ta đã học ở môn `xác suất thống kê`, được sử dụng để đánh giá sự tin cậy của kết quả và đưa ra quyết định về việc có chấp nhận hay từ chối giả thuyết. Thông thường, ngưỡng đáng chú ý (đặt trước) được chọn và so sánh với p-value để đưa ra quyết định.
- Vậy câu hỏi đặt ra là `Tại sao trong các phương pháp trên lại sử dụng p-value cho từng feature thì có thể xác định mô hình tốt nhất nên có feature ấy hay không?` Lý do sử dụng p-value là để đánh giá mức độ ảnh hưởng của từng `biến độc lập` đến `biến phụ thuộc`. Nếu một biến có `p-value` lớn hơn `ngưỡng đáng chú ý`, có nghĩa là không có đủ bằng chứng để cho rằng biến đó có ảnh hưởng đáng kể đến biến phụ thuộc. Do đó, việc loại bỏ biến đó khỏi mô hình không gây mất mát đáng kể về khả năng dự đoán.

## Cách triển khai của từng phương pháp
- **All-in**
    - Đây không hẳn là một kĩ thuật/phương pháp vì nhiệm vụ của ta là chỉ đơn giản dùng tất cả các ` biến độc lập` trong mô hình. Đây là điều không nên làm vì ta biết sẽ có những `garrbage feature` làm cho mô hình của ta tệ đi, **tuy nhiên** trong đồ án này thì việc sử dụng toàn bộ ` biến độc lập` mà ta có trong một mô hình sẽ cho ta một kết quả khả quan. Vì vậy mô hình sử dụng toàn bộ các ` biến độc lập` cũng không phải là một lựa chọn quá tồi đối với bộ dataset trong đồ án này.
- **Backward elimination**
    - Bước 1: Xác định ngưỡng giá trị tối đa của `p-value` hay còn gọi là `significance value (SL)`. Nó là một ngưỡng quyết định để xác định xem giả thuyết không có tác động có thể bị bác bỏ hay không. Các giá trị phổ biến cho mức ý nghĩa bao gồm 0.05 (5%) và 0.01 (1%).
    - Bước 2: Xây dựng mô hình hồi quy tuyến tính với tất cả các ` biến độc lập` có trong dữ liệu.
    - Bước 3: Sử dụng các công cụ, thư viện có sẵn (cụ thể là `statsmodels.api`) để tính toán ra `p-value` của các ` biến độc lập`. Lấy ` biến độc lập` có giá trị `p-value` cao nhất. Nếu nó cao hơn `SL` thì sang **bước 4**. Nếu không thì sẽ kết thúc quá trình.
    - Bước 4: Loại bỏ ` biến độc lập` này ra khỏi mô hình để có mô hình mới. Tính giá trị `mae (Mean Absolute Error)` của mô hình mới này. Nếu nhỏ hơn `mae` của mô hình cũ thì ta sẽ giữ lại mô hình cũ và kết thúc phương pháp. Nếu không thì ta sẽ quay lại **Bước 2** với các ` biến độc lập` còn được giữ lại.
- **Forward elimination**
    - Bước 1: Xác định ngưỡng giá trị tối đa của `p-value` hay còn gọi là `significance value (SL)` như ở phương pháp `Backward Elimination` để xác định xem ` biến độc lập` nào sẽ được thêm vào mô hình. 
    - Bước 2: Xây dựng các mô hình với chỉ một ` biến độc lập` lấy ra từ tập hợp các biến độc lập. Mô hình nào có ` biến độc lập` có `p-value` nhỏ nhất sẽ được giữ lại.
    - Bước 3: Lần lượt bổ sung một biến độc lập từ tập các `biến độc lập còn lại` vào mô hình đang có.
    - Bước 4: Xét `p-value` của các `biến độc lập` mới thêm vô trong mô hình mới. Tìm ra `p-value` nhỏ nhất. Nếu nó nhỏ hơn cả giá trị `SL` và `mae` của mô hình mới này nhỏ hơn `mae` của mô hình trước đó thì sẽ chính thức thêm ` biến độc lập` mới này vào mô hình hiện tại còn không thì sẽ không thêm ` biến độc lập` này vào và loại bỏ nó hoàn toàn khỏi tập các ` biến độc lập còn lại` , sau đó sẽ quay lại **Bước 3**. Nếu không sẽ giữ lại mô hình cũ (mô hình trước khi thêm ` biến độc lập` mới) và kết thúc quá trình.


## Tìm mô hình

### Phương pháp All-in

In [12]:
# Trình bày các phần tìm ra mô hình 
first_model_1d = train.columns.to_list()[:-1]

$$
\text{Model1:} \\ \text{Salary} = w1 \times Gender + w2 \times 10percentage + w3 \times 12percentage + w4 \times CollegeTier \\ + w5 \times Degree + w6 \times collegeGPA + w7 \times CollegeCityTier + w8 \times English + w9 \times Logical \\ + w10 \times Quant + w11 \times Domain w12 \times ComputerProgramming + w13 \times ElectronicsAndSemicon \\ + w14 \times ComputerScience + w15 \times MechanicalEngg + w16 \times ElectricalEngg + w17 \times TelecomEngg \\ + w18 \times CivilEngg + w19 \times conscientiousness + w20 \times agreeableness + w21 \times extraversion \\ + w22 \times nueroticism + w23 \times openesstoexperience
$$

### Phương pháp Backward Elimination

In [13]:
# Trình bày các phần tìm ra mô hình 
def calc_mae_current_model(X_train, y_train, X_test, y_test):
    model = OLSLinearRegression().fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return mae(y_test, y_pred)
# BACKWARD ELIMINATION
def backward_elimination(train, test, SL_value):
    selected_features = train.columns.to_list()[:-1]
    best_mae = calc_mae_current_model(train.iloc[:, :-1], train.iloc[:, -1], test.iloc[:, :-1], test.iloc[:, -1])
    while True:
        X_train = train.loc[:, selected_features]
        y_train = train.iloc[:,-1]
        y_test = test.iloc[:,-1]
        # dùng để giả sử loại biến độc lập đang xét khỏi mô hình
        temp_selected_features = selected_features.copy()
        
        regressor_OLS = sm.OLS(endog = y_train, exog =X_train).fit()
        p_values = regressor_OLS.pvalues.to_numpy()
        index_max_pvalue = np.argmax(p_values)
        if p_values[index_max_pvalue] > SL_value:
            # giả sử loại biến độc lập đang xét khỏi mô hình
            temp_selected_features.pop(index_max_pvalue)
            mae_new_model = calc_mae_current_model(train.loc[:,temp_selected_features], y_train, test.loc[:, temp_selected_features], y_test)
            # nếu tốt hơn thì sẽ loại bỏ chính thức nó khỏi mô hình và tiếp tục thuật toán
            if mae_new_model < best_mae:
                best_mae = mae_new_model
                selected_features.pop(index_max_pvalue)
            # nếu không tốt hơn thì sẽ không loại bỏ khỏi mô hình và kết thúc thuật toán
            else:
                return selected_features
        else: 
            return selected_features 
      

second_model_1d = backward_elimination(train=train, test=test, SL_value=0.05)

$$
\text{Model2:} \\
\text{Salary} = w1 \times Gender + w2 \times 10percentage + w3 \times 12percentage + w4 \times CollegeTier + w5 \times collegeGPA \\ + w6 \times English + w7 \times Logical + w8 \times Quant + w9 \times Domain + w10 \times ComputerProgramming \\+ w11 \times ElectronicsAndSemicon + w12 \times ComputerScience + w13 \times ElectricalEngg \\+ w14 \times TelecomEngg + w15 \times CivilEngg + w16 \times conscientiousness + w17 \times agreeableness \\+ w18 \times extraversion + w19 \times nueroticism + w20 \times openesstoexperience
$$

### Phương pháp Forward Elimination

In [14]:
# Trình bày các phần tìm ra mô hình
# FORWARD ELIMINATION
def forward_elimination(train, test, SL_value):
    remain_features = train.columns.to_list()[:-1]
    selected_features = []
    number_feature = 0
    best_mae = np.finfo(np.float32).max
    while True:
        p_values = []
        # lần lượt thêm 1 biến độc lập vào mô hình
        for feature in remain_features:
            temp_selected_features = selected_features.copy()
            temp_selected_features.append(feature)
            X_train = train.loc[:, temp_selected_features]
            y_train = train.iloc[:,-1]
            # lấy p-value của các biến độc lập vừa mới thêm vào
            regressor_OLS = sm.OLS(endog = y_train, exog =X_train).fit()
            p_value = regressor_OLS.pvalues.to_numpy()[number_feature]
            p_values.append(p_value)
        # lấy ra p-value nhỏ nhất trong các p-value của các biến độc lập vừa thêm vô
        index_min_pvalue = np.argmin(p_values)
        # nhỏ hơn SL_value thì tiếp tục
        if p_values[index_min_pvalue] < SL_value:
            selected_features.append(remain_features[index_min_pvalue])
            remain_features.pop(index_min_pvalue)
            current_mae = calc_mae_current_model(train.loc[:, selected_features], train.iloc[:,-1], test.loc[:, selected_features], test.iloc[:, -1])
            # nếu thêm vào rồi mà làm mô hình tệ hơn thì sẽ không thêm và loại nó ra khỏi hẳn "remain_features"
            if current_mae < best_mae:
                best_mae = current_mae
                number_feature += 1
            else:
                selected_features.pop(-1)
        # lớn hơn thì không thêm vào và dừng thuật toán
        else: return selected_features


third_model_1d = forward_elimination(train=train, test=test, SL_value=0.05)

$$
\text{Model3:} \\
\text{Salary} = w1 \times Gender + w2 \times Logical + w3 \times Quant + w4 \times ComputerScience \\ + w5 \times ComputerProgramming + w6 \times 10percentage + w7 \times CollegeTier + w8 \times collegeGPA \\+ w9 \times ElectricalEngg + w10 \times Domain + w11 \times conscientiousness + w12 \times ElectronicsAndSemicon \\ + w13 \times 12percentage
$$

## Thử nghiệm, so sánh các mô hình

In [15]:
# Phần code cho yêu cầu 1d
# Tìm ra mô hình tốt nhất (tự thiết kế bởi sinh viên)
# In ra các kết quả cross-validation như yêu cầu
features_1d = train.columns.to_list()
models_1d = [first_model_1d, second_model_1d, third_model_1d]
kfcv_er_1d = k_folds_cross_validation(k=8, features=features_1d,models=models_1d ,data=train)
kfcv_er_1d.shuffle_data()
kfcv_er_1d.split_to_k_folds()
kfcv_er_1d.cross_validation()

avg_mae_of_each_model, index_best_feature = kfcv_er_1d.best_model()
for i in range(avg_mae_of_each_model.shape[0]):
    print(f"Mae cho mô hình thứ {i+1} từ k-fold Cross Validation (lấy trung bình): {avg_mae_of_each_model[i]}")
print(f"Mô hình tốt nhất là mô hình thứ {index_best_feature+1}")

Mae cho mô hình thứ 1 từ k-fold Cross Validation (lấy trung bình): 110180.47330236036
Mae cho mô hình thứ 2 từ k-fold Cross Validation (lấy trung bình): 109938.75401221815
Mae cho mô hình thứ 3 từ k-fold Cross Validation (lấy trung bình): 111724.38017610791
Mô hình tốt nhất là mô hình thứ 2


- Báo cáo **3 kết quả tương ứng cho 3 mô hình** từ k-fold Cross Validation (lấy trung bình)
	
	<center>

	| STT | Mô hình  | MAE  |
	|:---:|:-----------------------:|:----:|
	|  1  | Mô hình có từ phương pháp All-in			        |110180.473      |
	|  **2**  | **Mô hình có từ phương pháp Backward Elimination**		            |**109938.754**      |
	|  3  | Mô hình có từ phương pháp Forward Elimination		            |111724.380     |

	</center>

In [16]:
# Huấn luyện lại mô hình my_best_model trên toàn bộ tập huấn luyện
# Lấy các dữ liệu cần thiết
X_train_1d = train.loc[:, models_1d[index_best_feature]].to_numpy().reshape(-1, len(models_1d[index_best_feature]))
y_train_1d = train.iloc[:, -1]
X_test_1d = test.loc[:, models_1d[index_best_feature]].to_numpy().reshape(-1, len(models_1d[index_best_feature]))
y_test_1d = test.iloc[:, -1]
# Huấn luyện lại mô hình best_skill_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện  
my_best_model = OLSLinearRegression().fit(X_train_1d, y_train_1d)
# In ra các hệ số tương ứng vừa huấn luyện được cho từng feature
params_1d = my_best_model.get_params()
for i in range(len(params_1d)):
    print(f"{models_1d[index_best_feature][i]}: {params_1d[i]}")
# Tính giá trị dự đoán của tập kiểm tra dựa theo mô hình vừa có được
y_pred_1d = my_best_model.predict(X_test_1d)

Gender: -24571.327475195576
10percentage: 880.0123438131209
12percentage: 1176.5137520883886
CollegeTier: -80548.71185860362
collegeGPA: 1778.113134013691
English: 153.54362237985413
Logical: 124.19292899390946
Quant: 103.58383206952433
Domain: 29779.497519190405
ComputerProgramming: 68.79185254935842
ElectronicsAndSemicon: -56.398554800186325
ComputerScience: -180.66800163061887
ElectricalEngg: -159.57461792015997
TelecomEngg: -70.04197991232131
CivilEngg: 153.80910624305528
conscientiousness: -19782.870196187047
agreeableness: 15167.239957385136
extraversion: 5031.589018552328
nueroticism: -10670.593842283619
openess_to_experience: -5619.403797379307


In [17]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình my_best_model
mae_1d = mae(y=y_test_1d, y_hat=y_pred_1d)
print("Độ lỗi tuyệt đối trung bình (MAE - Mean Absolute Error) của mô hình ở câu 1d: ", mae_1d)

Độ lỗi tuyệt đối trung bình (MAE - Mean Absolute Error) của mô hình ở câu 1d:  101421.37866107898


Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012):
$$\text{Salary} = -24571.327 \times Gender + 880.012 \times 10percentage + 1176.514 \times 12percentage \\-80548.712 \times CollegeTier + 1778.113 \times collegeGPA + 153.544 \times English + 124.193 \times Logical \\+ 103.584 \times Quant + 29779.497 \times Domain + 68.792 \times ComputerProgramming \\-56.398 \times ElectronicsAndSemicon -180.668 \times ComputerScience -159.574 \times ElectricalEngg \\ -70.042 \times TelecomEngg 153.809 \times CivilEngg -19782.870 \times conscientiousness \\+ 15167.240 \times agreeableness + 5031.589 \times extraversion -10670.594 \times nueroticism \\-5619.404 \times openesstoexperience. $$
Độ lỗi tuyệt đối trung bình MAE trên tập kiểm tra của mô hình tốt nhất ở câu 1d (làm tròn đến 3 chữ số thập phân):
$$ \text{MAE} =  101421.379$$